In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import datetime as dt
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataset = pd.read_csv('../input/coursera-course-dataset/coursea_data.csv');
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset.describe().transpose()

# > **CONTENT BASED FILTERING**

**CARA 1
CONTENT : JUDUL**

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(dataset['course_title'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(dataset.index, index=dataset['course_title']).drop_duplicates()

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    course_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return dataset['course_title'].iloc[course_indices]

In [ ]:
get_recommendations('AI for Medical Diagnosis')

CARA 2

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize

In [ ]:
bow = CountVectorizer(stop_words="english", tokenizer=word_tokenize)
bank = bow.fit_transform(dataset.course_title)

In [ ]:
idx = 0
content = dataset.loc[idx,'course_title']
content

In [ ]:
code = bow.transform([content])
code

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

dist = cosine_distances(code,bank)
dist

In [ ]:
rec_idx = dist.argsort()[0,1:11]
rec_idx

In [ ]:
dataset.loc[rec_idx]

CARA 3

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
# Apply clean_data function to your features.
features = ['course_organization', 'course_Certificate_type', 'course_difficulty']

for feature in features:
    dataset[feature] = dataset[feature].apply(clean_data)

In [ ]:
def create_soup(x):
    return ''.join(x['course_title']) + ' '  + ''.join(x['course_organization']) + ' '  + ''.join(x['course_difficulty'])
dataset['soup'] = dataset.apply(create_soup, axis=1)
dataset.head(10)

In [ ]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(dataset['soup'])
count_matrix

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
cosine_sim2

In [ ]:
# Reset index of our main DataFrame and construct reverse mapping as before
dataset = dataset.reset_index()
indices = pd.Series(dataset.index, index=dataset['course_title'])

HASIL REKOMENDASI

In [ ]:
rec = get_recommendations('AI for Medical Diagnosis', cosine_sim2)
rec